In [1]:
import torch
torch.cuda.is_available()

True

In [53]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sent_train.csv')
valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sent_valid.csv')

In [54]:
tweets = pd.concat([train, valid], ignore_index=True)

In [55]:
tweets.loc[tweets['label'] == 0, 'label_original'] = 'Bearish'
tweets.loc[tweets['label'] == 1, 'label_original'] = 'Bullish'
tweets.loc[tweets['label'] == 2, 'label_original'] = 'Neutral'

In [11]:
tweets.head(-5)

,text,label,label_original
0,$BYND - JPMorgan reels in expectations on Beyo...,0,Bearish
1,$CCL $RCL - Nomura points to bookings weakness...,0,Bearish
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0,Bearish
3,$ESS: BTIG Research cuts to Neutral https://t....,0,Bearish
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0,Bearish
...,...,...,...
11921,Lithium Americas and Gulf Resources among Ener...,2,Neutral
11922,Marvell Technology (MRVL) Gains As Market Dips...,2,Neutral
11923,Peloton Interactive among consumer gainers; Py...,2,Neutral
11924,Sarepta Therapeutics leads healthcare gainers;...,2,Neutral


In [56]:
from datasets import Dataset
dataset_tweets = Dataset.from_pandas(tweets)
split_tweets = dataset_tweets.train_test_split(test_size=0.2, shuffle=True, seed=42)

In [57]:
split_tweets['train']

Dataset({
    features: ['text', 'label', 'label_original'],
    num_rows: 9544
})

In [58]:
small_tweet_train_dataset = split_tweets["train"]
small_tweet_test_dataset = split_tweets["test"]

In [59]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")

In [60]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
   
tokenized_train_tweet = small_tweet_train_dataset.map(preprocess_function, batched=True)
tokenized_test_tweet = small_tweet_train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9544 [00:00<?, ? examples/s]

Map:   0%|          | 0/9544 [00:00<?, ? examples/s]

In [61]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [62]:
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert",num_labels=3)

In [63]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [68]:
from transformers import TrainingArguments, Trainer
 
repo_name = "dipterv-finbert"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=20,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_tweet,
   eval_dataset=tokenized_test_tweet,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [69]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1943 in _inner_training_loop     │
│                                                                                                  │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│   1942 │   │   │   │   │   and not is_torch_tpu_available()                                      │
│ ❱ 1943 │   │   │   │   │   and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))          │
│   1944 │   │   │   │   ):                                                                        │
│   1945 │   │   │   │   │   # if loss is nan or inf simply add the average of previous logged lo  │
│   1946 │   │   │   │   │   tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [67]:
trainer.evaluate()

{'eval_loss': 0.3006907105445862,
 'eval_accuracy': 0.8944886839899413,
 'eval_f1': 0.8940045812405191,
 'eval_runtime': 31.6776,
 'eval_samples_per_second': 301.285,
 'eval_steps_per_second': 18.846,
 'epoch': 1.0}

In [ ]:
trainer.save_model("/content/drive/MyDrive/my_model")